# Neurosur workshop 2021

## Tutorial 2 - Autoencoders
#### Creator: Manos Theodosis

---

## Tutorial objectives

In this notebook we will learn a few things about autoencoders
- Implement a linear autoencoder and contrast it with PCA.
- Implement a nonlinear (and sparse) autoencoder.
- Implement a deep autoencoder.
- Compare the different autoencoders.

---
## Imports and helper functtions

Execute the following cells that import packages needed for this notebook.

In [ ]:
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
plt.set_cmap("gray")

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchsummary

from torch.utils.data import DataLoader, Dataset, TensorDataset
from torchvision.utils import make_grid, save_image
import torchvision.datasets as ds

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
def load_mnist(datadir="./data_cache"):
    train_ds = ds.MNIST(root=datadir, train=True, download=True, transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                             ]))
    test_ds = ds.MNIST(root=datadir, train=False, download=True, transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                             ]))
    def to_xy(dataset):
        Y = dataset.targets.long()
        X = dataset.data.view(dataset.data.shape[0], -1) / 255.0
        return X, Y

    X_tr, Y_tr = to_xy(train_ds)
    X_te, Y_te = to_xy(test_ds)
    mean = X_tr.mean(dim=0)
    X_tr -= mean
    X_te -= mean
    return X_tr, Y_tr, X_te, Y_te

def make_loader(dataset, batch_size=128, num_workers=4):
    return torch.utils.data.DataLoader(dataset, batch_size=batch_size, num_workers=num_workers, shuffle=True, pin_memory=True)

## Exercise 1: Linear autoencoders

As we've seen, autoencoders map us from the data domain to a latent representation. For this exercise, we will create a linear autoencoder and we will compare the latent representation with that of PCA.

### 1.1 Create the `LinearAutoencoder` class
In the next cell, fill in the code for the linear autoencoder.

**Note**: for this class, we will not include learnable biases in the linear layers, in order to be able to compare and contrast with PCA. Feel free to experiment with adding biases to this class, or removing them from the rest of the classes we will create.

In [ ]:
class LinearAutoencoder(nn.Module):
    def __init__(
        self,
        input_size,
        latent_size,
    ):
        super(LinearAutoencoder, self).__init__()

        self.input_size = input_size
        self.latent_size = latent_size
        
        # create the encoder and the decoder
        self.encoder = ...
        self.decoder = ...

    def forward(self, data):
        # encode the input to the latent space
        latent = ...
        
        # decode the latent representation back into the input space
        recons = ...
        return recons

### 1.2 Load MNIST
Let's load MNIST using the provided `load_mnist()` function and keep $5,000$ images to train our autoencoder. When creating data loaders, use a batch size of $256$ (to make training faster). Let's also visualize some of the data to see what they look like.

In [ ]:
# load MNIST and keep a subset
X_tr, Y_tr, X_te, Y_te = load_mnist()

num_train = 5000
indexes = ...

X_tr = X_tr[indexes]
Y_tr = Y_tr[indexes]

# generate loaders
batch_size = 256
train_dl = make_loader(TensorDataset(X_tr, Y_tr), batch_size=batch_size)
test_dl = make_loader(TensorDataset(X_te, Y_te), batch_size=batch_size)

# plot some of the data
indexes_train = ...
fig, axes = plt.subplots(10, 10, figsize=(8,8))

idx = 0
for row in axes:
    for col in row:
        img = ...
        col.imshow(img)
        col.axis("off")
        idx += 1
plt.show()

### 1.3 Create a model
We will now create a model from the `LinearAutoencoder` class that we defined, and we'll print the summary in order to get a sense of how many parameters it has.

In [ ]:
input_size = 784
latent_size = 100
model = ...

In [ ]:
...

### 1.4 Train the model
We are now able to train the model we created. We will train using the `MSELoss()` for $20$ epochs, using a learning rate of $0.001$.

In [ ]:
num_epochs = 20
lr = 1e-3

opt = torch.optim.Adam(model.parameters(), lr=lr)
loss_func = ...

for epoch in range(num_epochs):
    print(f'Starting Epoch {epoch}')
    
    # training phase
    ...
    net_loss = 0.0
    n_total = 0
    for x, _ in tqdm(train_dl):
        x = x.to(device)
        
        # pass the data through the model
        out = ...
        loss = ...
        
        # backpropagate
        ...
        ...
        ...
        
        # keep loss statistics
        net_loss += loss.item() * len(x)
        n_total += len(x)
    train_loss = net_loss / n_total

    # evaluation phase
    ...
    net_loss = 0.0
    n_total = 0
    with torch.no_grad():
        for idx, (x, _) in enumerate(test_dl):
            x = x.to(device)

            # pass the data through the model
            out = ...
            loss = ...
            
            # keep loss statistics
            net_loss += loss.item() * len(x)
            n_total += len(x)
        test_loss = net_loss / n_total
    print(f'Epoch {epoch}:\t Train Loss: {train_loss:.3f}\t Test Loss: {test_loss:.3f}')

### 1.5 Visualize the ouputs
We have now trained our model and we are able to visualize its various outputs. We will focus on three things:
- We want to visualize the filters/weights that the network is learning.
- We want to visualize the latent representations of the data.
- We want to visualize the reconstruction, to make sure the network is working.

Fill in the code in the following three cells to generate the corresponding visualizations.

In [ ]:
# visualize weights
fig, axes = plt.subplots(10, 10, figsize=(8,8))

idx = 0
for row in axes:
    for col in row:
        img = ...
        col.imshow(img)
        col.axis("off")
        idx += 1
plt.show()

In [ ]:
# visualize latents
fig, axes = plt.subplots(10, 10, figsize=(8,8))
model_enc = ...
indexes_test = ...

idx = 0
for row in axes:
    for col in row:
        img = ...
        col.imshow(img)
        col.axis("off")
        idx += 1
plt.show()

In [ ]:
# visualize reconstructions
fig, axes = plt.subplots(10, 10, figsize=(8,8))
model_test = ...

idx = 0
for row in axes:
    for col in row:
        img = ...
        col.imshow(img)
        col.axis("off")
        idx += 1
plt.show()

### 1.6 Compare reconstructions with those of PCA
Since autoencoders are unsupervised techniques that can help in dimensionality reduction, it is natural to compare them with PCA. Run PCA below using `scikit-learn` and plot the reconstructions in order to compare them with the reconstructions of our autoencoder.

In [ ]:
# run PCA on the training data
pca = ...
pca.fit(X_tr)

# evaluate on the test data
pca_test = ...
inv_test = ...

# plot the reconstructions
fig, axes = plt.subplots(10, 10, figsize=(8,8))

idx = 0
for row in axes:
    for col in row:
        img = ...
        col.imshow(img)
        col.axis("off")
        idx += 1
plt.show()

It turns out that _linear_ autoencoders learn the principal components of the data, and are equivalent with PCA. In the following, we will start experimenting with _nonlinear_ autoencoders.

## Exercise 2: Nonlinear autoencoders
Since linear autoencoders are learning the principal components of the data, we will introduce nonlinearities to allow our models to be more expressive.

### 2.1 Create the `NonLinearAutoencoder` class
In the next cell, fill in the code for the nonlinear autoencoder.

**Note**: feel free to experiment with other nonlinearities (e.g. `PReLU`, `tanh`, `sigmoid` or others from [torch.nn.functional](https://pytorch.org/docs/stable/nn.functional.html)) and report your findings. Do all of them work equally well?

In [ ]:
class NonLinearAutoencoder(nn.Module):
    def __init__(
        self,
        input_size,
        latent_size,
    ):
        super(NonLinearAutoencoder, self).__init__()

        self.input_size = input_size
        self.latent_size = latent_size
    
        # create the encoder and the decoder
        self.encoder = ...
        self.decoder = ...
            
    def forward(self, data):
        # encode the input to the latent space
        latent = ...
        
        # decode the latent representation back into the input space
        recons = ...
        
        # output the latent representation too so we can use it for sparsity
        return recons, latent

### 2.2 Create the model
Create now an model from the `NonLinearAutoencoder` class and print it's summary. Since the only thing we changed is that we added nonlinearities, we expect the number of parameters to be the same.

In [ ]:
input_size = 784
latent_size = 100
model2 = ...

...

### 2.3 Train the model
Let's now train the model using the same parameters as before.

In [ ]:
num_epochs = 20
lr = 1e-3

opt = torch.optim.Adam(model2.parameters(), lr=lr)
loss_func = nn.MSELoss()

for epoch in range(num_epochs):
    print(f'Starting Epoch {epoch}')
    
    # training phase
    ...
    net_loss = 0.0
    n_total = 0
    for x, y in tqdm(train_dl):
        x = x.to(device)

        # pass the data through the model
        out, _ = ...
        loss = ...
        
        # backpropagate
        ...
        ...
        ...
        
        # keep loss statistics
        net_loss += loss.item() * len(x)
        n_total += len(x)
    train_loss = net_loss / n_total

    # evaluation phase
    ...
    net_loss = 0.0
    n_total = 0
    with torch.no_grad():
        for idx, (x, _) in enumerate(test_dl):
            x = x.to(device)

            # pass the data through the model
            out, _ = ...
            loss = ...
            
            # keep loss statistics
            net_loss += loss.item() * len(x)
            n_total += len(x)
        test_loss = net_loss / n_total
    print(f'Epoch {epoch}:\t Train Loss: {train_loss:.3f}\t Test Loss: {test_loss:.3f}')

### 2.4 Visualize the ouputs
As before, let's visualize the weights, latent representations, and reconstructions.

In [ ]:
# visualize weights
fig, axes = plt.subplots(10, 10, figsize=(8,8))

idx = 0
for row in axes:
    for col in row:
        img = ...
        col.imshow(img)
        col.axis("off")
        idx += 1
plt.show()

In [ ]:
# visualize latents
fig, axes = plt.subplots(10, 10, figsize=(8,8))
_, model_test = ...

idx = 0
for row in axes:
    for col in row:
        img = ...
        col.imshow(img)
        col.axis("off")
        idx += 1
plt.show()

In [ ]:
#visualize reconstructions
fig, axes = plt.subplots(10, 10, figsize=(8,8))
model_test, _ = ...

idx = 0
for row in axes:
    for col in row:
        img = ...
        col.imshow(img)
        col.axis("off")
        idx += 1
plt.show()

Our latent representations look a bit sparser than before, but the weights the network is learning are still not very interpretable. Let's try adding sparsity on the codes in order to enforce the autoencoder to learn more interpretable features.

### 2.4 Train with a sparse objective
Modify your training procedure from before to include a sparsity penalty. Set the regularization parameter `lam = 0.00125`.

**Note**: feel free to experiment with different values of `lam`. How does increasing this parameter affects the latent representations and the weights? What about decreasing it? Are the reconstructions still acceptable?

In [ ]:
num_epochs = 20
lr = 1e-3
lam = 0.00125

model2 = NonLinearAutoencoder(input_size, latent_size).to(device)

opt = torch.optim.Adam(model2.parameters(), lr=lr)
loss_func = nn.MSELoss()

for epoch in range(num_epochs):
    print(f'Starting Epoch {epoch}')
    
    # training phase
    ...
    net_loss = 0.0
    n_total = 0
    for x, _ in tqdm(train_dl):
        x = x.to(device)

        # pass the data through the model
        out, code = ...
        loss = ...
        
        # backpropagate
        ...
        ...
        ...
        
        # keep loss statistics
        net_loss += loss.item() * len(x)
        n_total += len(x)
    train_loss = net_loss / n_total

    # evaluation phase
    ...
    net_loss = 0.0
    n_total = 0
    with torch.no_grad():
        for idx, (x, _) in enumerate(test_dl):
            x = x.to(device)

            # pass the data through the model
            out, code = ...
            loss = ...
            
            # keep loss statistics
            net_loss += loss.item() * len(x)
            n_total += len(x)
        test_loss = net_loss / n_total
    print(f'Epoch {epoch}:\t Train Loss: {train_loss:.3f}\t Test Loss: {test_loss:.3f}')

### 2.5 Visualize the outputs
As before, visualize the outputs of the network.

In [ ]:
# visualize weights
fig, axes = plt.subplots(10, 10, figsize=(8,8))

idx = 0
for row in axes:
    for col in row:
        img = ...
        col.imshow(img)
        col.axis("off")
        idx += 1
plt.show()

In [ ]:
# visualize latents
fig, axes = plt.subplots(10, 10, figsize=(8,8))
_, model_test = ...

idx = 0
for row in axes:
    for col in row:
        img = ...
        col.imshow(img)
        col.axis("off")
        idx += 1
plt.show()

In [ ]:
#visualize reconstructions
fig, axes = plt.subplots(10, 10, figsize=(8,8))
model_test, _ = ...

idx = 0
for row in axes:
    for col in row:
        img = ...
        col.imshow(img)
        col.axis("off")
        idx += 1
plt.show()

We can see that the latent representations are significantly sparser now and the filters are starting to look a little bit more like digits, and the reconstructions are still reasonable. For the final part, let's make a deep autoencoder.

## Exercise 3: Deep autoencoders
Let's examine how the depth of the autoencoder affects

### 3.1 Create the `DeepAutoencoder` class
In the next cell, fill in the code for the deep autoencoder.

In [ ]:
class DeepAutoencoder(nn.Module):
    def __init__(
        self,
        input_size,
        firstlayer_size,
        seconlayer_size,
        thirdlayer_size,
    ):
        super(DeepAutoencoder, self).__init__()

        self.input_size = input_size
        self.firstlayer_size = firstlayer_size
        self.seconlayer_size = seconlayer_size
        self.thirdlayer_size = thirdlayer_size
    
        # create the encoder and the decoder
        self.encoder = nn.Sequential(
            ...
            nn.ReLU(),
            ...
            nn.ReLU(),
            ...
            nn.ReLU()
        )
        self.decoder = nn.Sequential(
            ...
            nn.ReLU(),
            ...
            nn.ReLU(),
            ...
            nn.ReLU(),
        )
            
    def forward(self, data):
        # encode the input to the latent space
        latent = ...
        
        # decode the latent representation back into the input space
        recons = ...
        
        # output the latent representation too so we can use it for sparsity
        return recons, latent

### 3.2 Train with a sparse objective
We will train the deep autoencoder directly using a sparse objective. Create a model from the `DeepAutoencoder` class with $400$, $200$, and $100$ sizes for the intermediate layers. Set the regularization parameter `lam = 0.00005` and use the same parameters as before for training.

In [ ]:
num_epochs = 20
lr = 1e-3
lam = 0.00005

input_size = 784
firstlayer_size = 400
seconlayer_size = 200
thirdlayer_size = 100
model3 = ...

opt = torch.optim.Adam(model3.parameters(), lr=lr)
loss_func = nn.MSELoss()

for epoch in range(num_epochs):
    print(f'Starting Epoch {epoch}')
    
    # training phase
    ...
    net_loss = 0.0
    n_total = 0
    for x, _ in tqdm(train_dl):
        x = x.to(device)

        # pass the data through the model
        out, code = ...
        loss = ...
        
        # backpropagate
        ...
        ...
        ...

        # keep loss statistics
        net_loss += loss.item() * len(x)
        n_total += len(x)
    train_loss = net_loss / n_total

    # evaluation phase
    ...
    net_loss = 0.0
    n_total = 0
    with torch.no_grad():
        for idx, (x, _) in enumerate(test_dl):
            x = x.to(device)

            # pass the data through the model
            out, code = ...
            loss = ...
            
            # keep loss statistics
            net_loss += loss.item() * len(x)
            n_total += len(x)
        test_loss = net_loss / n_total
    print(f'Epoch {epoch}:\t Train Loss: {train_loss:.3f}\t Test Loss: {test_loss:.3f}')

### 3.3 Visualize the ouputs
As before, let's visualize the weights, latent representations, and reconstructions.

In [ ]:
# visualize weights
fig, axes = plt.subplots(10, 10, figsize=(8,8))
weights = ...

idx = 0
for row in axes:
    for col in row:
        img = ...
        col.imshow(img)
        col.axis("off")
        idx += 1
plt.show()

In [ ]:
# visualize latents
fig, axes = plt.subplots(10, 10, figsize=(8,8))
_, model_test = ...

idx = 0
for row in axes:
    for col in row:
        img = ...
        col.imshow(img)
        col.axis("off")
        idx += 1
plt.show()

In [ ]:
# visualize reconstructions
fig, axes = plt.subplots(10, 10, figsize=(8,8))
model_test, _ = ...

idx = 0
for row in axes:
    for col in row:
        img = ...
        col.imshow(img)
        col.axis("off")
        idx += 1
plt.show()

We can see that the weights that we are learning right now look more aggresively like digits (albeit, still, very noisy).

# Wrap-up

In this notebook we learned about training autoencoders, and we examined different trade-offs as we are adding nonlinearities, sparsity, and depth.